In [1]:
import torch
import os
from dataset import CDEDataset
from torchvision import transforms
from torch.utils.data import DataLoader
import time
import numpy as np
import matplotlib.pyplot as plt

if torch.cuda.is_available(): 
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
device

device(type='cuda', index=0)

In [2]:
no_dilation_models = []
for i in range(7):
    no_dilation_models.append(torch.load(f"Models/dilationNone/conv_{i+1}_800.pt"))
    
dilation_2_models = []
for i in range(8):
    dilation_2_models.append(torch.load(f"Models/dilation2/dil2_{i+1}_800.pt"))
    
aspp_models = []
for i in range(7):
    aspp_models.append(torch.load(f"Models/aspp/aspp_{i+1}_800.pt"))

In [3]:
root_ims = '../CDE_Data/ShanghaiTech/ShanghaiTech/part_A/test_data/images'
root_ann = '../CDE_Data/ShanghaiTech/ShanghaiTech/part_A/test_data/density_gt'
im_list = os.listdir(root_ims)
test_SH_A = CDEDataset(im_list,root_ims,root_ann, transform  = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]), train =False)

root_ims = '../CDE_Data/ShanghaiTech/ShanghaiTech/part_B/test_data/images'
root_ann = '../CDE_Data/ShanghaiTech/ShanghaiTech/part_B/test_data/density_gt'
im_list = os.listdir(root_ims)
test_SH_B = CDEDataset(im_list,root_ims,root_ann, transform  = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]), train =False)

root_ims = '../CDE_Data/UCF_CC_50/images'
root_ann = '../CDE_Data/UCF_CC_50/density_gt'
im_list = os.listdir(root_ims)
test_UCF = CDEDataset(im_list,root_ims,root_ann, transform  = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ]), train =False)

In [4]:
test_SH_A_dataloader = DataLoader(test_SH_A, batch_size=1, shuffle=True)
test_SH_B_dataloader = DataLoader(test_SH_B, batch_size=1, shuffle=True)
test_UCF_dataloader = DataLoader(test_UCF, batch_size=1, shuffle=True)

test_dataloaders = [test_SH_A_dataloader, test_SH_B_dataloader, test_UCF_dataloader]

In [5]:
def evaluate(model,dataloaders):

    maes = []
    mses = []
    for loader in dataloaders:
        losses =[]

        model.eval()
        with torch.no_grad():    
            for i, data in enumerate(loader, 0):
                # evaluate the model on the test set
                inputs, labels, true_count = data


                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)

                pred_count = np.sum(outputs[0][0].cpu().detach().numpy())
                true_count = true_count.cpu().detach().numpy()

                losses.append(pred_count - true_count)

                print(i, end = " ")
        mae = np.mean(np.abs(losses))
        mse = np.sqrt(np.mean(np.square(losses)))
        maes.append(mae)
        mses.append(mse)
    return maes, mses
        

In [6]:
no_dil_maes = []
no_dil_mses = []

for model in no_dilation_models:
    maes, mses = evaluate(model, test_dataloaders)
    no_dil_maes.append(maes)
    no_dil_mses.append(mses)
    
no_dil_maes = np.array(no_dil_maes)
no_dil_mses = np.array(no_dil_maes)
    
dil_2_maes = []
dil_2_mses = []

for model in dilation_2_models:
    maes, mses = evaluate(model, test_dataloaders)
    dil_2_maes.append(maes)
    dil_2_mses.append(mses)

dil_2_maes = np.array(dil_2_maes)
dil_2_mses = np.array(dil_2_mses)
    
aspp_maes = []
aspp_mses = []

for model in aspp_models:
    maes, mses = evaluate(model, test_dataloaders)
    aspp_maes.append(maes)
    aspp_mses.append(mses)
    
aspp_maes = np.array(aspp_maes)
aspp_mses = np.array(aspp_mses)   

0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 

In [8]:
aspp_maes

array([[120.81903076,  38.983675  , 950.10710144],
       [123.23551941,  52.90758228, 621.83308411],
       [ 82.04025269,  21.45450974, 694.24008179],
       [144.58102417,  23.0792942 , 805.4524231 ],
       [133.02142334,  28.56994486, 711.70791626],
       [112.3518219 ,  26.41638803, 756.69429016],
       [164.46046448,  19.9282465 , 765.34695435]])

In [9]:
dil_2_maes

array([[205.03204346,  34.22775173, 675.51594543],
       [         inf,          inf,          inf],
       [183.43681335,  70.24850082, 621.87406921],
       [138.30209351,  17.42902756, 655.72915649],
       [182.39247131,  20.5289135 , 651.93612671],
       [126.82597351,  75.07871246, 630.59580994],
       [107.56401062,  27.73271799, 211.41662598],
       [225.17709351,  14.88031769, 464.63659668]])

In [7]:
ex = test_UCF[5]
im = ex[0].unsqueeze(0).to(device)
im.shape

start = time.time()
a = modeltest(im)

print("inference time:", time.time() - start)
print("actual: ",ex[2])
print("predicted: ",np.sum(a[0][0].cpu().detach().numpy()))

fig, axs = plt.subplots(1, 3, figsize=(15,20))
axs[0].imshow(ex[0][0].detach().numpy())
axs[1].imshow(ex[1][0].detach().numpy())
axs[2].imshow(a[0][0].cpu().detach().numpy())
plt.show()

IndexError: list index out of range